# NLP: basic topic modelling for Twitter data
In this notebook, we are going to use our cleaned dataset that we have created in the second notebook.

We are going to do some basic Natural Language Processing (NLP) - we will create a topic model for the `extended_tweet_cleaned` column.

## Goal: 
Learn what topic modelling is (specifically LDA), apply topic modelling algoithms on different subsets of the input data, learn how to visualize topics.

## Introduction to LDA
[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)(Latent Dirichlet Allocation) is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar.(Wikipedia)


LDA divides documents (tweets) into topics(clusters). 
It assumes that:
 - every topic (e.g. sport) has some representative words (e.g soccer, play, ball. etc.), 
 - every document(tweet) has some representative topics(e.g sport, summer, etc.).

Topics can interfere: the same word can belong to multiple topics.  
Documents (tweets) can have multiple topics.

### Topic modelling process:
 - Subset the data
 - Prepare data for topic modelling(tokenize, lemmatize)
 - Apply topic modelling
 - Vizualize and explore
 - Repeat if needed

#### Lets download the cleaned tiwitter data from object storage and display first 5 rows

Load python libraries first. Additional libraries that we are going to use in this notebook are: 
 - [gensim](https://pypi.org/project/gensim/) -  Python library for topic modelling
 - [nltk](https://www.nltk.org/) - natural language toolkit, library to work with language
 - [pyLDAvis](https://pypi.org/project/pyLDAvis/) - library for interactive topic model visualization.

In [ ]:
try:
    import urllib.request
except ImportError:
    !pip install  --user  urllib
    import urllib.request
    
try:
    import pandas as pd
except ImportError:
    !pip install  --user  pandas
    import pandas as pd

try:
    import gensim
    import gensim.corpora as corpora
    from gensim.utils import simple_preprocess
except:
    !pip install  --user  gensim
    import gensim
    import gensim.corpora as corpora
    from gensim.utils import simple_preprocess
    
try:
    import pyLDAvis.gensim
except:
    !pip install  --user pyldavis
    import pyLDAvis.gensim

try:
    import nltk
    from nltk.corpus import wordnet as wn
except:
    !pip install  --user  nltk
    import nltk
    from nltk.corpus import wordnet as wn

#### Download the cleaned dataset from object store and display first 5 rows
There is a copy of cleaned dataset saved to object store. You can download it or use your local copy, created in the second notebook.
We reading csv file into pandas dataframe and printing first 5 rows.

In [ ]:
target_url="https://swift-yeg.cloud.cybera.ca:8080/v1/AUTH_233e84cd313945c992b4b585f7b9125d/geeky-summit/tweets_cleaned1.csv"
file_name="tweets_cleaned1.csv"
urllib.request.urlretrieve(target_url, file_name) ## comment out this line to use your local copy 

tweets = pd.read_csv(file_name,parse_dates=['created_at_date'])  ## reading 'created_at_date' column as timestamp
pd.set_option('max_colwidth', 20)
tweets.head()

### Step1 Subsetting the data
To make LDA algoritm work faster we will use only subset of the data instead of the entire dataset.  
Let's try subsetting by the day first:

In [ ]:
tweets_subset_nov5=tweets.loc[tweets["created_at_date"].dt.day==5]

### Step2 Preparing data for LDA

#### [Tokenization](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html) is the task of chopping sentence up into pieces, called tokens, (in our case we will split tweets into words).  
First we transform extended_tweet_cleaned column into a List of Strings (tweets).   
We are only going to be working with this column  - so don't need the entire dataset.

In [ ]:
data=tweets_subset_nov5["extended_tweet_cleaned"].tolist()
print (data[1]) ###printing 1st element of the list

Second we will go through every String(tweet) in  the List and transform it into another List: list of words(tokens).  
We are going to use [gensim.utils.simple_preprocess](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess) function, it converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long.

In [ ]:
def string_to_words(sentences):
    for sentence in sentences:
        ## will ignore words shorter than 4 and longer than 15 characters
        yield(gensim.utils.simple_preprocess(sentence,min_len=4, max_len=15))  

data_tokens = list(string_to_words(data))

print(data_tokens[1]) ###printing 1st element of the list after tokenization

#### Removing [stowords](https://pythonspot.com/nltk-stop-words/) - removing words that don't add any value and can be excluded from ananlysis
NLTK libabry has some predefined stopwords, we will download them and examine:

In [ ]:
nltk.download('stopwords')

stop_words = set(nltk.corpus.stopwords.words('english'))
#stop_words.add('calgary')
print(stop_words)

#### [Lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)  - reducing word forms and unifying them to one main form.
For example:
>is $\Rightarrow$ be   
>car, cars $\Rightarrow$ car

We are going to use wn.morphy() function to get lemmas for words.


In [ ]:
nltk.download('wordnet')

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [ ]:
print(get_lemma("is"))
print(get_lemma("cars"))

In [ ]:
def tokens_to_lda(data_tokens):
    for data_token in data_tokens:
        tokens = [token for token in data_token if token not in stop_words]
        tokens = [get_lemma(token) for token in tokens]
        yield tokens
tokens = list(tokens_to_lda(data_tokens))

###printing 1st element of the list after removing stopwords and lemmatization
print(tokens[1]) 

#### Creating dictionary and corpus objects for LDA model
We will create dictionary and corpus for LDA model.  

**[Dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary)** - a mapping between words and their integer ids.  
**Corpus** -  list of (word_id, word_frequency) for every document.

We will be  using [corpora.Dictionary()](https://radimrehurek.com/gensim/corpora/dictionary.html)  and 
[doc2bow()](https://kite.com/python/docs/gensim.corpora.dictionary.Dictionary.doc2bow) methods.

In [ ]:
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]

print("First 10 words in the dictionary: ", list(dictionary.token2id)[:10])

print("First tweet, tokens: ",tokens[1])
print("First tweet, corpus: ",corpus[1])

### Step 3 Building LDA model with 7 topics and displaying top 8 words for every topic

We are going to use [gensim.models.ldamodel.LdaModel()](https://radimrehurek.com/gensim/models/ldamodel.html) function with predefined number of topics - 8 and print top 7 words in every topic.

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=7, 
                                           random_state=100)

topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)

### Step 4 Visualizing the model 
We are going to use [pyLDAvis.gensim.prepare] (https://pyldavis.readthedocs.io/en/latest/modules/API.html#pyLDAvis.prepare)

In [ ]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=True)
pyLDAvis.display(lda_display)

####  exercise: try different days, try modifying stop words, number of topics  and min_len /max_len in `gensim.utils.simple_preprocess`

##  Conclusion


In this notebook we introduced the idea of topic modeling on a corpus of text, and showed the results of a popular topic modeling algorithm, namely LDA. We also outlined some important pre-processing steps that one can take in order to prepare text data for analysis in a topic model. We outlined tokenization, lemmatization and the removal of stop words in order to simplify our corpus in order to simplify the corpus for our topic model. This notebook should prepare you for the next notebook which combines sentiment analysis and topic modeling to understand how people "feel" about a given topic.